# Recurrent Neural Network
시계열 데이터(오디오, 주가, 문장) 등 요소의 순서가 중요한 데이터는 데이터의 전후관계가 매우 중요하여 이를 반영하여야 한다.
RNN은 이전 상태(t-1)에서 나온 결과를 현재 상태(t)에도 반영하는 방법이다. 이전에 사왔던 hidden state를 특정 계속 반영해주면 t=4일 때 이전 상태에서 쓰였던 값들이 함께 반영되어 가중치를 업데이트 하게 된다. 마치 IIR 필터와 비슷하다.  
![](https://miro.medium.com/v2/resize:fit:1400/1*xn5kA92_J5KLaKcP7BMRLA.gif)

각 변수에 대한 설명은 아래와 같다.
* t — time step
* X — input
* h — hidden state
* length of X — size/dimension of input
* length of h — no. of hidden units. Note that different libraries call them differently, but they mean the same:
  * Keras — state_size ,units
  * PyTorch — hidden_size
  * TensorFlow — num_units
  
![](https://miro.medium.com/v2/resize:fit:1400/1*1aOMOtRPXu4Ln2WtrwuM1A.gif)

In [ ]:
encoded = tf.constant([[vocab[word] for word in sentence.split()]])
print(sentence, f"=> one-hot: {encoded}")

# 100차원으로 의미를 분산
layer_embedding = tf.keras.layers.Embedding(input_dim=len(vocab), output_dim=100)
embedded = layer_embedding(encoded)
print("Embedding Layer Table and distributed encoded sentence Shape", layer_embedding.weights[0].shape, embedded.shape)

layer_rnn1 = tf.keras.layers.SimpleRNN(units=10, return_sequences=True, use_bias=True)
layer_rnn1_out = layer_rnn1(embedded)
print("First RNN Layer weight and output shape:", layer_rnn1.weights[0].shape, layer_rnn1_out.shape)

from keras.utils import layer_utils
print(len(layer_rnn1.weights), layer_rnn1.weights[-1].shape)
print(f"trainable {layer_utils.count_params(layer_rnn1.trainable_weights)}")
print(f"trainable {layer_utils.count_params(layer_rnn1.non_trainable_weights)}")

위 코드에서 layer_rnn1의 총 파라미터는 1100개고 1000개는 trainable, 100개는 non-trainable이다.  
$recurrent_weights + input_weights + biases$ 이 총 파라미터 갯수가 된다. 먼저 recurrent_weights와 input_weight를 계산해본다.  
weights[0] 에는 input_weight인 1000개가 들어있다. 100개의 차원으로 분석하였으니 feature는 100이 되고 아웃풋 뉴런이 10개이니 100 x 10 하여 1000개가 된다.  
그리고 recurrent_weights는 이전 시퀀스에서 가지고 온 아웃풋 뉴런이 10개이므로 이전 뉴런과 새로이 쓰일 뉴런을 곱해 10x10 100개 된다. 만야 bias또한 사용하였다면 아웃풋 뉴런에 붙을 것이므로 10 x 1이되서 1110개가 된다.

아래 예시는 입력이 3이고 중간 뉴런이 4개일 경우이다.
recurrent weight는 4*4으로 16개, input_weight는 4*3으로 12개, 바이어스는 4*1이되어 4개로 총 32개가 trainable parameter가 된다.
![](https://i.stack.imgur.com/71An9.png)

## 장단점
RNN의 경우 입력 데이터가 길면 길수록 concatenate를 해가면서 뒤로 전달할 때 앞단의 정보가 뒤로 갈수록 옅어지는 문제가 있다.
!()[https://research.aimultiple.com/wp-content/uploads/2021/08/rnn-text.gif]
### 기울기 소실과 기울기 폭발
역전파를 하면서 RNN역시 기울기 소실이 일어나는데 RNN은 기울기 폭발도 일어난다. 이전 상태로 역전파를 하면서 기울기를 구할 때 연쇄법칙에 의해 앞에 오는 미분계수값이 1보다 크게 되면 뒤로 점점 발산해버리는 현상이다. 두 경우 모두 가중치가 업데이트가 안되거나 아예 튀어버리므로 학습이 불가능하다.